In [1]:
%pip install networkx==2.5
%pip install  dgl -f https://data.dgl.ai/wheels/repo.html
%pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html
!pip install ipython-autotime
%load_ext autotime



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.0 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 21.6 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.9/112.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 47.8 MB/s eta 0:00:00
     ━━━━━━

In [2]:
# get data from drive
from google.colab import drive

drive.mount('/content/gdrive')
drive_path = "/content/gdrive/MyDrive/fyp_data/"

Mounted at /content/gdrive
time: 17.9 s (started: 2023-10-15 07:41:00 +00:00)


Mounted at /content/gdrive


In [3]:
attack_name = "xssreflected"
attack_type = "benign"

time: 1.94 ms (started: 2023-10-15 07:41:18 +00:00)


In [4]:
# import required packages
import matplotlib.pyplot as plt
import networkx as nx
import gzip, pickle
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
from dgl.data import DGLDataset
import torch as th
import json
from collections import defaultdict
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import OrderedDict
from dgl.nn.pytorch import GraphConv
import os
from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from dgl.dataloading import GraphDataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# set constant seed for consistency across multiple settings
torch.manual_seed(42)
dgl.seed(42)

device_name = torch.device('cpu')
print("Using {}.".format(device_name))

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Using cpu.
time: 7.46 s (started: 2023-10-15 07:41:18 +00:00)


In [ ]:
node_types = ['argv', 'block', 'address', 'task', 'process_memory', 'file', 'socket', 'pipe', 'iattr', 'link', 'machine', 'path', 'shm']
edge_types = ['wasAssociatedWith', 'used', 'wasGeneratedBy', 'wasInformedBy', 'wasDerivedFrom', 'wasAttributedTo', 'actedOnBehalfOf']

node_features_count = len(node_types)
edge_features_count = len(edge_types)

targets = np.array(node_types)
nodelabelEnc = LabelEncoder()
new_target = nodelabelEnc.fit_transform(targets)
nodeencoder = OneHotEncoder(sparse_output=False)
nodeencoder.fit(new_target.reshape(-1, 1))
nodeencoder = nodeencoder

targets = np.array(edge_types)
edgelabelEnc = LabelEncoder()
new_target = edgelabelEnc.fit_transform(targets)
edgeencoder = OneHotEncoder(sparse_output=False)
edgeencoder.fit(new_target.reshape(-1, 1))
edgeencoder = edgeencoder


def one_hot_node(data):
  new_target = nodelabelEnc.transform(np.array(data))
  return torch.from_numpy(nodeencoder.transform(new_target.reshape(-1, 1)))

def one_hot_edge(data):
  new_target = edgelabelEnc.transform(np.array(data))
  return torch.from_numpy(edgeencoder.transform(new_target.reshape(-1, 1)))


class ProvenanceDataset(DGLDataset):
    def __init__(self):
      super().__init__(name='provenance')

    def read_graph(self, file_name):
      graph_raw = json.load(open(file_name,"r"))

      list_of_nodes = defaultdict(set)

      node_types = {}
      edge_types = []

      out_edges = []
      in_edges = []

      for key, value in graph_raw.items():
        node1_type, edge_type, node2_type = key.split("-")
        node1_index, node2_index = value
        for i in node1_index:
          node_types[i] = node1_type
          edge_types.append(edge_type)
        for i in node2_index:
          node_types[i] = node2_type


        out_edges = out_edges + node1_index
        in_edges = in_edges + node2_index

      node_types = OrderedDict(sorted(node_types.items()))
      number_of_edges = list(node_types.keys())[-1] + 1

      for i in range(number_of_edges):
        if i not in node_types:
          node_types[i] = 'task'

      out_edges = th.tensor(out_edges)
      in_edges = th.tensor(in_edges)

      g = dgl.graph((out_edges, in_edges))

      g.ndata['attr'] = (one_hot_node(list(node_types.values())))
      g.edata['attr'] = (one_hot_edge(edge_types))

      g = dgl.remove_self_loop(g)
      g = dgl.add_self_loop(g)

      return g

    def process(self):
        self.graphs = []
        self.labels = []

        # benign graph ID
        for graph_id in tqdm(range(1, 1001)):
          g = self.read_graph(f"{drive_path}benign_full/graph{graph_id}.json")
          self.graphs.append(g)
          self.labels.append(0)

        # benign graph ID
        for graph_id in tqdm(range(1, 1001)):
          g = self.read_graph(f"{drive_path}xssreflected_benign_1000/graph{graph_id}.json")
          self.graphs.append(g)
          self.labels.append(0)

        # Convert the label list to tensor for saving.
        self.labels = torch.LongTensor(self.labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = ProvenanceDataset()
graph, label = dataset[0]
print(graph, label, graph.device, label.device)

100%|██████████| 1000/1000 [06:03<00:00,  2.75it/s]

Graph(num_nodes=4610, num_edges=13828,
      ndata_schemes={'attr': Scheme(shape=(13,), dtype=torch.float64)}
      edata_schemes={'attr': Scheme(shape=(7,), dtype=torch.float64)}) tensor(0) cpu cpu
time: 12min 49s (started: 2023-10-14 16:43:47 +00:00)


In [ ]:
print(graph.number_of_nodes())

4006
time: 684 µs (started: 2023-10-14 09:58:45 +00:00)


In [ ]:
print(graph.all_edges())

(tensor([   1,   39,   50,  ..., 4003, 4004, 4005]), tensor([   0,   40,   51,  ..., 4003, 4004, 4005]))
time: 5.75 ms (started: 2023-10-14 10:00:15 +00:00)


In [ ]:
import json
st = set()
for graph_id in tqdm(range(1, 501)):
  x = f"{drive_path}attack_2/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.update(keysList)
for graph_id in tqdm(range(1, 501)):
  x = f"{drive_path}attack/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.update(keysList)
for graph_id in tqdm(range(1, 1001)):
  x = f"{drive_path}xssreflected_attack_1000/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.update(keysList)

print(st)
print("attack: " + str(len(st)))


b = set()
for graph_id in tqdm(range(1, 1001)):
  x = f"{drive_path}xssreflected_benign_1000/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  b.update(keysList)

for graph_id in tqdm(range(1, 1001)):
  x = f"{drive_path}benign_full/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  b.update(keysList)
print(b)
print("benign: " + str(len(b)))

100%|██████████| 1000/1000 [00:17<00:00, 58.59it/s]


{'task-used-pipe', 'socket-wasDerivedFrom-address', 'socket-wasDerivedFrom-socket', 'task-used-block', 'file-wasDerivedFrom-pipe', 'machine-wasAssociatedWith-task', 'process_memory-wasDerivedFrom-argv', 'process_memory-wasDerivedFrom-file', 'task-used-file', 'block-wasDerivedFrom-block', 'shm-wasGeneratedBy-task', 'pipe-wasDerivedFrom-pipe', 'socket-wasDerivedFrom-task', 'file-wasGeneratedBy-task', 'iattr-wasGeneratedBy-task', 'link-wasGeneratedBy-task', 'file-wasDerivedFrom-file', 'pipe-wasDerivedFrom-process_memory', 'task-used-process_memory', 'process_memory-wasGeneratedBy-task', 'file-wasDerivedFrom-process_memory', 'link-wasDerivedFrom-link', 'process_memory-wasDerivedFrom-path', 'file-wasDerivedFrom-socket', 'file-wasDerivedFrom-path', 'pipe-wasGeneratedBy-task', 'process_memory-wasDerivedFrom-process_memory', 'task-used-link', 'file-wasDerivedFrom-iattr', 'file-wasDerivedFrom-link', 'process_memory-wasDerivedFrom-pipe', 'shm-wasDerivedFrom-shm', 'block-wasGeneratedBy-task', 'so

100%|██████████| 1000/1000 [00:14<00:00, 69.06it/s]

{'task-used-pipe', 'socket-wasDerivedFrom-address', 'socket-wasDerivedFrom-socket', 'task-used-block', 'file-wasDerivedFrom-pipe', 'machine-wasAssociatedWith-task', 'process_memory-wasDerivedFrom-argv', 'process_memory-wasDerivedFrom-file', 'task-used-file', 'block-wasDerivedFrom-block', 'pipe-wasDerivedFrom-pipe', 'socket-wasDerivedFrom-task', 'file-wasGeneratedBy-task', 'iattr-wasGeneratedBy-task', 'file-wasDerivedFrom-file', 'pipe-wasDerivedFrom-process_memory', 'task-used-process_memory', 'process_memory-wasGeneratedBy-task', 'file-wasDerivedFrom-process_memory', 'link-wasDerivedFrom-link', 'process_memory-wasDerivedFrom-path', 'file-wasDerivedFrom-path', 'pipe-wasGeneratedBy-task', 'process_memory-wasDerivedFrom-process_memory', 'task-used-link', 'file-wasDerivedFrom-iattr', 'file-wasDerivedFrom-link', 'process_memory-wasDerivedFrom-pipe', 'block-wasGeneratedBy-task', 'socket-wasGeneratedBy-task', 'task-wasInformedBy-task', 'task-used-socket'}
benign: 32
time: 1min 2s (started: 20

In [8]:
import json
st = []
for graph_id in tqdm(range(1, 501)):
  x = f"{drive_path}attack_2/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.append(len(keysList))
for graph_id in tqdm(range(1, 501)):
  x = f"{drive_path}attack/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.append(len(keysList))
for graph_id in tqdm(range(1, 1001)):
  x = f"{drive_path}xssreflected_attack_1000/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  st.append(len(keysList))

print(st)
print("attack: " + str(sum(st)/len(st)))


b = []
for graph_id in tqdm(range(1, 1001)):
  x = f"{drive_path}xssreflected_benign_1000/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  b.append(len(keysList))

for graph_id in tqdm(range(1, 1001)):
  x = f"{drive_path}benign_full/graph{graph_id}.json"
  with open(x, 'r') as fh:
    dt = json.load(fh)
  keysList = list(dt.keys())
  b.append(len(keysList))
print(b)
print("benign: " + str(sum(b)/len(b)))

100%|██████████| 1000/1000 [05:53<00:00,  2.83it/s]


[23, 20, 19, 18, 21, 19, 21, 23, 23, 22, 22, 19, 21, 21, 22, 20, 18, 21, 22, 16, 21, 20, 21, 20, 25, 20, 23, 19, 21, 20, 22, 22, 20, 24, 20, 21, 21, 22, 22, 23, 22, 19, 21, 18, 24, 24, 23, 22, 23, 21, 23, 23, 20, 21, 22, 18, 22, 22, 21, 24, 24, 21, 21, 22, 23, 23, 22, 23, 22, 21, 24, 23, 23, 18, 19, 22, 21, 21, 24, 23, 23, 23, 20, 19, 19, 21, 22, 21, 21, 25, 23, 25, 21, 23, 20, 19, 21, 22, 21, 24, 22, 19, 24, 22, 20, 18, 21, 23, 21, 22, 23, 22, 22, 24, 19, 20, 24, 22, 23, 23, 23, 23, 22, 24, 20, 24, 21, 22, 21, 19, 22, 22, 24, 21, 23, 23, 21, 22, 24, 22, 21, 24, 20, 22, 21, 23, 18, 20, 23, 21, 21, 21, 21, 24, 23, 19, 19, 25, 20, 22, 23, 21, 19, 23, 22, 22, 22, 27, 22, 21, 22, 19, 21, 19, 22, 27, 20, 20, 21, 21, 21, 19, 20, 21, 22, 19, 19, 18, 19, 21, 19, 22, 20, 18, 20, 21, 22, 21, 19, 19, 19, 18, 23, 19, 19, 18, 20, 19, 18, 20, 20, 21, 18, 19, 20, 22, 22, 19, 19, 19, 18, 19, 18, 20, 21, 19, 18, 18, 18, 19, 19, 22, 19, 18, 19, 18, 23, 23, 20, 23, 19, 18, 19, 19, 19, 19, 19, 19, 18, 19,

100%|██████████| 1000/1000 [06:13<00:00,  2.68it/s]

[23, 20, 18, 18, 18, 19, 20, 19, 22, 21, 23, 21, 16, 18, 18, 18, 14, 18, 24, 19, 23, 21, 20, 21, 20, 23, 20, 19, 19, 19, 17, 18, 19, 18, 17, 18, 22, 19, 19, 21, 23, 21, 19, 19, 19, 31, 20, 18, 17, 15, 19, 19, 16, 22, 21, 23, 23, 24, 17, 20, 19, 19, 22, 20, 20, 19, 22, 21, 20, 20, 18, 20, 22, 19, 22, 18, 21, 21, 20, 18, 20, 22, 19, 18, 19, 20, 21, 21, 19, 18, 19, 18, 18, 19, 19, 18, 19, 19, 19, 20, 19, 18, 19, 19, 20, 18, 18, 18, 18, 18, 20, 21, 19, 18, 19, 20, 23, 19, 20, 16, 18, 18, 21, 19, 18, 15, 18, 20, 19, 20, 20, 21, 21, 22, 20, 20, 19, 17, 21, 19, 18, 21, 23, 19, 23, 19, 18, 18, 16, 15, 18, 20, 18, 22, 21, 21, 19, 20, 19, 18, 16, 18, 18, 18, 18, 20, 18, 15, 18, 19, 22, 22, 20, 18, 20, 17, 18, 18, 18, 17, 19, 16, 18, 23, 18, 18, 19, 19, 18, 20, 19, 19, 21, 18, 19, 18, 17, 17, 18, 21, 18, 20, 19, 17, 18, 17, 17, 15, 16, 22, 22, 18, 21, 20, 19, 20, 19, 20, 18, 21, 21, 23, 20, 21, 19, 19, 19, 20, 18, 20, 19, 19, 21, 22, 23, 19, 19, 16, 21, 22, 23, 21, 19, 18, 19, 18, 19, 20, 22, 23,